In [1]:
!pip install streamlit
!npm install -g localtunnel
!pip install streamlit requests pandas plotly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼Requirement already satisfied: streamlit in /usr/local/lib/python3.11/dist-packages (1.43.0)


In [2]:
%%writefile app_ui.py
import streamlit as st
import requests
import json
import time
import os
import tempfile

# ✅ FastAPI Backend URL (Update with the correct Ngrok URL if running remotely)
BACKEND_URL = os.getenv("BACKEND_URL", "https://1963-35-185-253-183.ngrok-free.app")  # Update this URL if needed

# ✅ API Endpoints
ANALYZE_API_URL = f"{BACKEND_URL}/analyze_legal_document"
VIDEO_ANALYZE_API_URL = f"{BACKEND_URL}/analyze_legal_video"
LEGAL_QA_API_URL = f"{BACKEND_URL}/legal_chatbot"
# Endpoints for risk visualizations
RISK_BAR_API_URL = f"{BACKEND_URL}/download_risk_chart"
RISK_PIE_API_URL = f"{BACKEND_URL}/download_risk_pie_chart"
RISK_RADAR_API_URL = f"{BACKEND_URL}/download_risk_radar_chart"
RISK_TREND_API_URL = f"{BACKEND_URL}/download_risk_trend_chart"
INTERACTIVE_RISK_API_URL = f"{BACKEND_URL}/interactive_risk_chart"

# ✅ Configure Streamlit UI
st.set_page_config(page_title="AI-Powered Legal Assistant", layout="wide")

# ✅ Sidebar Navigation
st.sidebar.title("📌 Navigation")
menu = st.sidebar.radio("Select an option", ["🏠 Home", "📑 Document Analyzer", "🎥 Video Analyzer", "🤖 Legal Q&A"])

# ✅ Home Page
if menu == "🏠 Home":
    st.title("📜 AI-Powered Legal & Video Assistant")
    st.markdown("👋 Welcome to your AI-powered legal assistant. Choose an option from the sidebar.")

# ✅ Legal Document Analyzer
elif menu == "📑 Document Analyzer":
    st.title("📑 Legal Document Analyzer")
    uploaded_file = st.file_uploader("📂 Upload a Legal Document (PDF)", type=["pdf"])

    if uploaded_file:
        st.success("✅ File uploaded successfully! Analyzing...")

        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
            temp_file.write(uploaded_file.getbuffer())
            temp_file_path = temp_file.name

        files = {"file": open(temp_file_path, "rb")}
        try:
            with st.spinner("⏳ Processing your document... Please wait..."):
                response = requests.post(ANALYZE_API_URL, files=files, timeout=180)
                response.raise_for_status()
                result = response.json()

            st.success("✅ Document Analysis Completed!")

            # ✅ Retrieve and display Task ID
            task_id = result.get("task_id", "N/A")
            st.sidebar.write(f"🆔 **Task ID:** `{task_id}`")

            # ✅ Display Results
            st.subheader("📑 Document Summary")
            summary_text = result.get("summary", "⚠️ No summary available.")
            st.write(summary_text)

            # ✅ Legal Risk Assessment (Basic Scores)
            st.subheader("⚠️ Legal Risk Assessment")
            risk_scores = result.get("risk_scores", {})
            if risk_scores:
                st.json(risk_scores)
            else:
                st.warning("⚠️ No risk assessment data available.")

            # ✅ Detailed Risk Information (with contextual summaries)
            st.subheader("⚠️ Detailed Risk Information")
            detailed_risk = result.get("detailed_risk", {})
            if detailed_risk:
                for risk_term, info in detailed_risk.items():
                    st.markdown(f"### {risk_term}")
                    st.write(info)
            else:
                st.write("⚠️ No detailed risk information available.")

            # ✅ Additional Risk Visualizations
            st.subheader("📊 Additional Risk Visualizations")
            chart_option = st.selectbox("Select Chart Type",
                                         ["Bar Chart", "Pie Chart", "Radar Chart", "Trend Chart", "Interactive Chart"])
            chart_url = ""
            if chart_option == "Bar Chart":
                chart_url = RISK_BAR_API_URL
            elif chart_option == "Pie Chart":
                chart_url = RISK_PIE_API_URL
            elif chart_option == "Radar Chart":
                chart_url = RISK_RADAR_API_URL
            elif chart_option == "Trend Chart":
                chart_url = RISK_TREND_API_URL
            elif chart_option == "Interactive Chart":
                chart_url = INTERACTIVE_RISK_API_URL

            try:
                chart_response = requests.get(chart_url)
                if chart_response.status_code == 200:
                    if chart_option == "Interactive Chart":
                        st.components.v1.html(chart_response.text, height=600)
                    else:
                        # Save image to temporary file and display
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as chart_file:
                            chart_file.write(chart_response.content)
                            chart_file_path = chart_file.name
                        st.image(chart_file_path, caption=f"📊 {chart_option}")
                        os.remove(chart_file_path)
                else:
                    st.warning("⚠️ Risk visualization not available.")
            except requests.exceptions.RequestException:
                st.warning("⚠️ Could not load risk visualization.")

            # ✅ Contract Clauses
            st.subheader("📌 Detected Contract Clauses")
            clauses = result.get("clauses_detected", [])
            if clauses:
                for clause in clauses:
                    st.write(f"- **{clause['type']}** (Confidence: {clause['confidence']:.2f})")
            else:
                st.write("⚠️ No significant contract clauses detected.")

        except requests.exceptions.RequestException as e:
            st.error(f"❌ API Error: {e}")
        finally:
            os.remove(temp_file_path)

# ✅ Video Analyzer
elif menu == "🎥 Video Analyzer":
    st.title("🎬 Video Content Analyzer")
    uploaded_video = st.file_uploader("📂 Upload a Video File", type=["mp4", "avi", "mov", "mpeg4"])

    if uploaded_video:
        st.success("✅ Video uploaded successfully! Analyzing...")

        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_file:
            temp_file.write(uploaded_video.getbuffer())
            temp_file_path = temp_file.name

        files = {"file": open(temp_file_path, "rb")}
        try:
            with st.spinner("⏳ Processing your video... Please wait..."):
                response = requests.post(VIDEO_ANALYZE_API_URL, files=files, timeout=300)
                response.raise_for_status()
                result = response.json()

            st.success("✅ Video Analysis Completed!")

            # ✅ Retrieve and display Task ID
            task_id = result.get("task_id", "N/A")
            st.sidebar.write(f"🆔 **Task ID:** `{task_id}`")

            # ✅ Display Transcript
            st.subheader("📜 Video Transcript")
            transcript = result.get("transcript", "⚠️ No transcript available.")
            st.write(transcript)

            # ✅ Display Summary
            st.subheader("📑 Video Summary")
            summary = result.get("summary", "⚠️ No summary available.")
            st.write(summary)

            # ✅ Legal Risk Assessment (if available)
            st.subheader("⚠️ Legal Risk Assessment")
            risk_scores = result.get("risk_scores", {})
            if risk_scores:
                st.json(risk_scores)
            else:
                st.warning("⚠️ No risk assessment data available.")

            # ✅ Detailed Risk Information (if available)
            st.subheader("⚠️ Detailed Risk Information")
            detailed_risk = result.get("detailed_risk", {})
            if detailed_risk:
                for risk_term, info in detailed_risk.items():
                    st.markdown(f"### {risk_term}")
                    st.write(info)
            else:
                st.write("⚠️ No detailed risk information available.")

            # ✅ Contract Clauses
            st.subheader("📌 Detected Contract Clauses")
            clauses = result.get("clauses_detected", [])
            if clauses:
                for clause in clauses:
                    st.write(f"- **{clause['type']}** (Confidence: {clause['confidence']:.2f})")
            else:
                st.write("⚠️ No significant contract clauses detected.")

            # ✅ Provide Download Option for Transcript
            transcript_path = result.get("transcript_path")
            if transcript_path:
                st.download_button(label="📥 Download Transcript", data=transcript, file_name="video_transcript.txt")

        except requests.exceptions.RequestException as e:
            st.error(f"❌ API Error: {e}")
        finally:
            os.remove(temp_file_path)

# ✅ Legal Q&A
elif menu == "🤖 Legal Q&A":
    st.title("🤖 Legal Question Answering")
    st.write("💡 Ask a legal question based on a previously analyzed document or video.")

    task_id = st.text_input("🆔 Enter Task ID from document or video analysis:")
    question = st.text_input("💬 Ask your legal question:")

    if st.button("🔍 Get Answer"):
        if task_id.strip() and question.strip():
            with st.spinner("⏳ Processing your question... Please wait..."):
                try:
                    response = requests.post(
                        LEGAL_QA_API_URL,
                        data={"query": question, "task_id": task_id},
                        timeout=60
                    )
                    response.raise_for_status()
                    qa_response = response.json().get("response", "⚠️ No answer available.")

                    st.subheader("💡 AI's Response")
                    st.success(qa_response)

                except requests.exceptions.RequestException as e:
                    st.error(f"❌ API Error: {e}")
        else:
            st.warning("⚠️ Please enter both Task ID and a question.")


Writing app_ui.py


In [3]:
!cloudflared tunnel run

/bin/bash: line 1: cloudflared: command not found


In [4]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/


In [5]:
!pkill -9 streamlit


In [ ]:



# ✅ Run Streamlit and Expose it via Cloudflare Tunnel
!streamlit run app_ui.py & sleep 5 && cloudflared tunnel --url http://localhost:8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.93.237:8501

2025-03-06T09:50:49Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-03-06T09:50:49Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-03-06T09:50:55Z INF +--------------------------------------------------------------------------------------------+
2025-03-06T09:50:55Z INF 